In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import os

dataset_size = [50]   #Defien the dataset size, You can define this as an array if you want to generate datasets with different sizes
directory_txt = r'/Users/janithwanniarachchi/Downloads/Chama Simulations'  # Output file save location
directory = r'/Users/janithwanniarachchi/Downloads/Chama Simulations/D21 PBF SS316.npy'  # Update this path
def compute_odf(euler_angles, bins=10):
    """
    Computes the ODF for a given set of Euler angles using histograms.
    """
    odf = [np.histogram(euler_angles[:, i], bins=bins, density=True)[0] for i in range(3)]
    return odf

def optimal_clusters(data, max_clusters=75):
    """
    Determine the optimal number of clusters for KMeans clustering.
    """
    wcss = []
    for i in range(1, max_clusters+1, 5):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(data)
        wcss.append(kmeans.inertia_)
    changes = [(wcss[i] - wcss[i-1]) / wcss[i-1] for i in range(1, len(wcss))]
    start = 0
    for idx, change in enumerate(changes):
        if abs(change) < 0.10:
            start = (idx + 1) * 5
            break
    optimal_clusters = max_clusters
    for i in range(start, max_clusters+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(data)
        current_wcss = kmeans.inertia_
        change = (current_wcss - wcss[-1]) / wcss[-1]
        if abs(change) < 0.01:
            optimal_clusters = i
            break
        wcss.append(current_wcss)
    return optimal_clusters


features = np.load(directory)
# Extract the base file name (with extension)
filename = os.path.basename(directory)
filename_without_extension = os.path.splitext(filename)[0]
bestNumClusters = optimal_clusters(features)
print(f"K-means stabilized with {bestNumClusters} clusters for file {filename}")

for numTotalPoints in dataset_size:
    txt_output_filepath = os.path.join(directory_txt, f'{filename_without_extension}_{numTotalPoints}.txt')
    if os.path.exists(txt_output_filepath):
        print(f"File {txt_output_filepath} already exists. Skipping K-means clustering for this dataset size.")
        continue

    previous_odf = None
    prev_odf_difference = None
    converged = False
    iteration_count = 0
    min_iterations = 3
    max_iterations = 50

    while not converged:
        kmeans = KMeans(n_clusters=bestNumClusters, n_init=100, random_state=0).fit(features)
        bestIdx = kmeans.labels_

        # Adjust the number of points per cluster
        numPointsPerCluster = np.zeros(bestNumClusters, dtype=int)
        clusterSizes = np.array([len(features[bestIdx == i]) for i in range(bestNumClusters)])
        proportionalSizes = (clusterSizes / clusterSizes.sum()) * numTotalPoints
        numPointsPerCluster = np.ceil(proportionalSizes).astype(int)

        # Correct the total to match numTotalPoints exactly
        while numPointsPerCluster.sum() != numTotalPoints:
            difference = numTotalPoints - numPointsPerCluster.sum()
            indices = numPointsPerCluster.argsort()[::-1] if difference < 0 else numPointsPerCluster.argsort()
            for i in indices:
                if difference == 0:
                    break
                if difference > 0 or (difference < 0 and numPointsPerCluster[i] > 1):
                    numPointsPerCluster[i] += np.sign(difference)
                    difference -= np.sign(difference)

        representative_idx = []
        for i in range(bestNumClusters):
            clusterData = np.where(bestIdx == i)[0]
            if len(clusterData) > 0:
                distances = cdist(features[clusterData], features[clusterData])
                avgDistances = np.mean(distances, axis=1)
                densities = 1.0 / avgDistances
                densities /= densities.sum()
                sample_idx = np.random.choice(clusterData, size=numPointsPerCluster[i], replace=False, p=densities)
                representative_idx.extend(sample_idx)

        representative_data = features[representative_idx]
        current_odf = compute_odf(representative_data)

        if previous_odf is not None:
            odf_difference = sum(np.linalg.norm(current - prev) for current, prev in zip(current_odf, previous_odf))
            if prev_odf_difference is not None and iteration_count > min_iterations:
                rel_change = abs((odf_difference - prev_odf_difference) / prev_odf_difference)
                if rel_change < 0.1:
                    converged = True
            prev_odf_difference = odf_difference

        previous_odf = current_odf
        iteration_count += 1
        if iteration_count >= max_iterations:
            converged = True

        if converged:
            np.savetxt(txt_output_filepath, representative_data, delimiter='\t', fmt='%f')
            print(f"Data for {numTotalPoints} points saved to {txt_output_filepath}")
            break

print("Finished generating the files")


K-means stabilized with 44 clusters for file D21 PBF SS316.npy
